# Credit Card Default Prediction

The goal is to predict the likelihood of a client defaulting on their credit loans by crediting a credit score prediction model. 

By the end of this notebook exercise, we hope to have answered the following questions:
1. How does the probability of default payment vary by categories of different demographic variables?
2. Which variables are the strongest predictors of default payments?

# Dataset

Dataset contains 25 variables:

- **ID**: ID of each client
- **LIMIT_BAL**: Amount of given credit in NT dollars (includes individual and family/supplementary credit)
- **SEX**: Gender (1=male, 2=female)
- **EDUCATION**: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- **MARRIAGE**: Marital status (1=married, 2=single, 3=others)
- **AGE**: Age in years
- **PAY_0**: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
- **PAY_2**: Repayment status in August, 2005 (scale same as above)
- **PAY_3**: Repayment status in July, 2005 (scale same as above)
- **PAY_4**: Repayment status in June, 2005 (scale same as above)
- **PAY_5**: Repayment status in May, 2005 (scale same as above)
- **PAY_6**: Repayment status in April, 2005 (scale same as above)
- **BILL_AMT1**: Amount of bill statement in September, 2005 (NT dollar)
- **BILL_AMT2**: Amount of bill statement in August, 2005 (NT dollar)
- **BILL_AMT3**: Amount of bill statement in July, 2005 (NT dollar)
- **BILL_AMT4**: Amount of bill statement in June, 2005 (NT dollar)
- **BILL_AMT5**: Amount of bill statement in May, 2005 (NT dollar)
- **BILL_AMT6**: Amount of bill statement in April, 2005 (NT dollar)
- **PAY_AMT1**: Amount of previous payment in September, 2005 (NT dollar)
- **PAY_AMT2**: Amount of previous payment in August, 2005 (NT dollar)
- **PAY_AMT3**: Amount of previous payment in July, 2005 (NT dollar)
- **PAY_AMT4**: Amount of previous payment in June, 2005 (NT dollar)
- **PAY_AMT5**: Amount of previous payment in May, 2005 (NT dollar)
- **PAY_AMT6**: Amount of previous payment in April, 2005 (NT dollar)
- **default.payment.next.month**: Default payment (1=yes, 0=no)
Inspiration

# Exploratory Data Analysis
The goal of EDA is to uncover patterns, relationships, anomalies, and trends with the dataset. These discoveries provide insights that guides further analysis and decision-making. 

## Understand the data
- get a good understanding of the data: such as number of observations,features, and data types
- identify the target variable (variable which we want to predict) and understand its significance

### Import Libraries

In [2]:
import pandas as pd 
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [3]:
# read in the dataset
df = pd.read_csv(r"../data/raw/UCI_Credit_Card.csv")

### Data Size
Check how many observations does the dataset contain?


In [5]:
# check the shape of the dataset
df.shape # return the number of rows and columns as a tuple
print(f"The dataset has - {df.shape[0]} rows and {df.shape[1]} columns.")

The dataset has - 30000 rows and 25 columns.


### Data Preview
What does the dataset look like? 

In [6]:
# show the first five observations
df.head() # displays the first 5 rows of the dataset by default. 

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


### Data Types
What type of information is stored in each column? 

In [7]:
# return the data type information of each column
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

The dataset is composed mainly of int64s and float64s. The dataframe using 5.7mb of memory. We can optimise this by adjusting the capacity (data type) used to store each column. This is done by checking the data range of each column and changing the capacity used to store them. 


In [8]:
# rename the target column 
df = df.rename(columns={
    "default.payment.next.month":"def_pay"
})

In [9]:
# look at the memory used by each column
df.memory_usage(deep=True)

Index           132
ID           240000
LIMIT_BAL    240000
SEX          240000
EDUCATION    240000
MARRIAGE     240000
AGE          240000
PAY_0        240000
PAY_2        240000
PAY_3        240000
PAY_4        240000
PAY_5        240000
PAY_6        240000
BILL_AMT1    240000
BILL_AMT2    240000
BILL_AMT3    240000
BILL_AMT4    240000
BILL_AMT5    240000
BILL_AMT6    240000
PAY_AMT1     240000
PAY_AMT2     240000
PAY_AMT3     240000
PAY_AMT4     240000
PAY_AMT5     240000
PAY_AMT6     240000
def_pay      240000
dtype: int64

In [10]:
# get the statistical distribution of the dataset.
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID,30000.0,15000.500000,8660.398374,1.0,7500.75,15000.5,22500.25,30000.0
LIMIT_BAL,30000.0,167484.322667,129747.661567,10000.0,50000.00,140000.0,240000.00,1000000.0
SEX,30000.0,1.603733,0.489129,1.0,1.00,2.0,2.00,2.0
EDUCATION,30000.0,1.853133,0.790349,0.0,1.00,2.0,2.00,6.0
MARRIAGE,30000.0,1.551867,0.521970,0.0,1.00,2.0,2.00,3.0
AGE,30000.0,35.485500,9.217904,21.0,28.00,34.0,41.00,79.0
PAY_0,30000.0,-0.016700,1.123802,-2.0,-1.00,0.0,0.00,8.0
PAY_2,30000.0,-0.133767,1.197186,-2.0,-1.00,0.0,0.00,8.0
PAY_3,30000.0,-0.166200,1.196868,-2.0,-1.00,0.0,0.00,8.0
PAY_4,30000.0,-0.220667,1.169139,-2.0,-1.00,0.0,0.00,8.0


From the output above, the majority of numeric columns do not need to be stored with int64 datatype. Why? Their values are well under the max capacity required for an 1nt64. We can store in a datatype that is more close to their capacity. Also, some values are unsigned. Therefore no need for storage of negative ranges.  

In [11]:
# change to appropriate data type - uint8
df["AGE"] = df["AGE"].astype("uint8")
df["SEX"] = df["SEX"].astype("uint8")
df["EDUCATION"] = df["EDUCATION"].astype("uint8")
df["MARRIAGE"] = df["MARRIAGE"].astype("uint8")
df["def_pay"] = df["def_pay"].astype("uint8")

In [12]:
# change to appropriate data type - uint8
temp_list = [0, 2, 3, 4, 5, 6]
for i in temp_list:
    df[f"PAY_{i}"] = df[f"PAY_{i}"].astype("int8")

In [13]:
# change to appropriate data type - float32
temp_list = [1, 2, 3, 4, 5, 6]
for i in temp_list:
    df[f"PAY_AMT{i}"] = df[f"PAY_AMT{i}"].astype("float32")

In [14]:
# check memory usage after datatype changes
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         30000 non-null  int64  
 1   LIMIT_BAL  30000 non-null  float64
 2   SEX        30000 non-null  uint8  
 3   EDUCATION  30000 non-null  uint8  
 4   MARRIAGE   30000 non-null  uint8  
 5   AGE        30000 non-null  uint8  
 6   PAY_0      30000 non-null  int8   
 7   PAY_2      30000 non-null  int8   
 8   PAY_3      30000 non-null  int8   
 9   PAY_4      30000 non-null  int8   
 10  PAY_5      30000 non-null  int8   
 11  PAY_6      30000 non-null  int8   
 12  BILL_AMT1  30000 non-null  float64
 13  BILL_AMT2  30000 non-null  float64
 14  BILL_AMT3  30000 non-null  float64
 15  BILL_AMT4  30000 non-null  float64
 16  BILL_AMT5  30000 non-null  float64
 17  BILL_AMT6  30000 non-null  float64
 18  PAY_AMT1   30000 non-null  float32
 19  PAY_AMT2   30000 non-null  float32
 20  PAY_AM

- managed to reduce the memory used by the dataframe from 5.7mb to 2.8mb, a 68% difference

### Missing Values
Check for any missing or duplicate values in the dataset.

In [15]:
# check for missing values in the dataset
df.isna().sum()

ID           0
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
def_pay      0
dtype: int64

In [16]:
# check for data duplicates
print(f"Number of duplicate values {df.duplicated().sum()}.")

Number of duplicate values 0.


### Correlation Analysis
Important check - how does each dataset variable relate to other dataset variable. 

In [17]:
# calculate the correlation matrix
corr = df.corr()
# extract the target variable - def_pay
next_month_default_corr = corr["def_pay"].sort_values()
next_month_default_corr

LIMIT_BAL   -0.153520
PAY_AMT1    -0.072929
PAY_AMT2    -0.058579
PAY_AMT4    -0.056827
PAY_AMT3    -0.056250
PAY_AMT5    -0.055124
PAY_AMT6    -0.053183
SEX         -0.039961
MARRIAGE    -0.024339
BILL_AMT1   -0.019644
BILL_AMT2   -0.014193
BILL_AMT3   -0.014076
ID          -0.013952
BILL_AMT4   -0.010156
BILL_AMT5   -0.006760
BILL_AMT6   -0.005372
AGE          0.013890
EDUCATION    0.028006
PAY_6        0.186866
PAY_5        0.204149
PAY_4        0.216614
PAY_3        0.235253
PAY_2        0.263551
PAY_0        0.324794
def_pay      1.000000
Name: def_pay, dtype: float64

In [19]:
# plot the correlation heatmap
fig = px.imshow(corr, text_auto=True)
fig.update_layout(
    height=900,
    width=950
)
fig.show()

Extract the relationship between each independent variable and the target variable. 

In [20]:
fig = go.Figure()
fig.add_trace(
    go.Bar(x=next_month_default_corr[:-1].index, y= next_month_default_corr[:-1].values, text=next_month_default_corr[:-1].values)
)
fig.update_layout(
      plot_bgcolor="white",
      height=900,
      width=950,
      margin={
          "l":25,
          "r":25,
          "b":25
      },
      title_text="Default Correlation Distribution <br>",
      title={
          "x":0.5,
          "xanchor":"center",
          "font":{
              "size":14
          }
      },
      xaxis_title="Independent Variables",
      showlegend=False
  )
fig.update_xaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.update_yaxes(
      showline = True,
      linewidth=1,
      linecolor="black"
  )
fig.show()